# 06_Geospatial_Analysis - Heatmap der Wohnungsangebote

## 🎯 **Ziel:**
Erstellung einer interaktiven Heatmap (Choropleth) von Berlin, die die Verteilung der Wohnungsangebote pro Ortsteil visualisiert.

### **Hauptfunktionen:**
- **Laden der Geodaten** für Berliner Ortsteile.
- **Laden des angereicherten Datasets** mit Wohnungsangeboten.
- **Aggregation der Angebote** pro Ortsteil.
- **Verknüpfung** der Angebotsdaten mit den Geodaten.
- **Erstellung einer interaktiven Folium-Karte** mit Tooltips für Detailinformationen.

### **Input:**
- `data/processed/berlin_housing_combined_enriched_final.csv`
- `data/raw/lor_ortsteile.geojson`

### **Output:**
- Interaktive HTML-Karte (`heatmap_offers_by_district.html`)

---
**Teil der modularen Analyse-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.0

## 1. Setup und Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import warnings
warnings.filterwarnings('ignore')

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"GeoPandas Version: {gpd.__version__}")
print(f"Folium Version: {folium.__version__}")

## 2. Lade Geodaten und angereicherte Angebotsdaten

In [ ]:
print("="*60)
print("LADE GEODATEN UND ANGEBOTSDATEN")
print("="*60)

# Lade GeoJSON-Datei mit den Ortsteil-Grenzen
geojson_path = 'data/raw/lor_ortsteile.geojson'
try:
    gdf_districts = gpd.read_file(geojson_path)
    print(f"✅ GeoJSON geladen: {len(gdf_districts)} Ortsteile")
except Exception as e:
    print(f"❌ Fehler beim Laden der GeoJSON-Datei: {e}")

# Lade das angereicherte Dataset
data_path = 'data/processed/berlin_housing_combined_enriched_final.csv'
try:
    df_offers = pd.read_csv(data_path)
    print(f"✅ Angereicherte Angebotsdaten geladen: {len(df_offers):,} Zeilen")
except Exception as e:
    print(f"❌ Fehler beim Laden der Angebotsdaten: {e}")

## 3. Aggregiere Angebotsdaten pro Ortsteil

In [ ]:
print("="*60)
print("AGGREGIERE ANGEBOTSDATEN PRO ORTSTEIL")
print("="*60)

# Berechne die Anzahl der Angebote pro Ortsteil
offers_by_district = df_offers['ortsteil_neu'].value_counts().reset_index()
offers_by_district.columns = ['ortsteil_neu', 'offer_count']

print(f"✅ Daten aggregiert: {len(offers_by_district)} Ortsteile mit Angeboten")
print("
Top 10 Ortsteile nach Angeboten:")
print(offers_by_district.head(10))

## 4. Verknüpfe Geodaten mit Angebotsdaten

In [ ]:
print("="*60)
print("VERKNÜPFE GEODATEN MIT ANGEBOTSDATEN")
print("="*60)

# Merge die Geodaten mit den aggregierten Angebotsdaten
gdf_merged = gdf_districts.merge(offers_by_district, left_on='OTEIL', right_on='ortsteil_neu', how='left')

# Fülle NaN-Werte in 'offer_count' mit 0 (für Ortsteile ohne Angebote)
gdf_merged['offer_count'] = gdf_merged['offer_count'].fillna(0).astype(int)

print(f"✅ Merge abgeschlossen. Spalten im GeoDataFrame: {list(gdf_merged.columns)}")
print(f"Anzahl der Ortsteile im GeoDataFrame: {len(gdf_merged)}")

## 5. Erstelle die interaktive Heatmap

In [ ]:
print("="*60)
print("ERSTELLE INTERAKTIVE HEATMAP")
print("="*60)

# Erstelle eine Folium-Karte, zentriert auf Berlin
m = folium.Map(location=[52.52, 13.405], zoom_start=10)

# Erstelle eine Choropleth-Karte
folium.Choropleth(
    geo_data=gdf_merged,
    name='choropleth',
    data=gdf_merged,
    columns=['OTEIL', 'offer_count'],
    key_on='feature.properties.OTEIL',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Anzahl der Wohnungsangebote'
).add_to(m)

# Füge Tooltips hinzu
folium.features.GeoJsonTooltip(
    fields=['OTEIL', 'offer_count'],
    aliases=['Ortsteil:', 'Angebote:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
).add_to(folium.features.GeoJson(gdf_merged))

# Speichere die Karte als HTML-Datei
output_map_path = 'heatmap_offers_by_district.html'
m.save(output_map_path)

print(f"✅ Interaktive Karte gespeichert unter: {output_map_path}")